<a href="https://colab.research.google.com/github/ProfesorCamacho/Analisis_Estados_Financieros/blob/main/Analisis_de_estados_financieros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import sqlite3
import pandas as pd

# Ruta a la base de datos SQLite
ruta_base_de_datos = 'income_statements_select_07_21_.db'

# Nombre de la tabla a leer
nombre_tabla = 'Edo_Resultados'

# Conectar a la base de datos SQLite
conexion = sqlite3.connect(ruta_base_de_datos)

# Leer la tabla en un DataFrame
df = pd.read_sql_query(f'SELECT * FROM {nombre_tabla}', conexion)

# Cerrar la conexión
conexion.close()

# Mostrar los primeros 5 registros del DataFrame
print(df.head())

                  date symbol reportedCurrency        cik  \
0  2015-03-31 00:00:00     AA              USD  1675149.0   
1  2015-06-30 00:00:00     AA              USD  1675149.0   
2  2015-09-30 00:00:00     AA              USD  1675149.0   
3  2015-12-31 00:00:00     AA              USD  1675149.0   
4  2016-03-31 00:00:00     AA              USD  1675149.0   

           fillingDate         acceptedDate  calendarYear period  \
0  2015-03-31 00:00:00  2015-03-31 00:00:00        2015.0     Q2   
1  2015-06-30 00:00:00  2015-06-30 00:00:00        2015.0     Q2   
2  2015-09-30 00:00:00  2015-09-30 00:00:00        2015.0     Q3   
3  2015-12-31 00:00:00  2015-12-31 00:00:00        2015.0     Q4   
4  2016-03-31 00:00:00  2016-03-31 00:00:00        2016.0     Q2   

        revenue  costOfRevenue  ...  incomeBeforeTaxRatio  incomeTaxExpense  \
0  3.105000e+09   2.291000e+09  ...              0.132689       186000000.0   
1  2.964000e+09   2.352000e+09  ...              0.005735        4

In [22]:
# Contamos cuantas obvservaciones tenemos por empresa
conteo_empresas = df['symbol'].value_counts()
conteo_empresas

symbol
TILE    75
ASGN    73
LH      69
PPBI    68
T       68
        ..
BBL      2
LEV      2
THCB     2
TSIA     2
TBB      1
Name: count, Length: 1130, dtype: int64

In [23]:
# Filtrar las empresas con al menos 30 observaciones
empresas_30_obs = conteo_empresas[conteo_empresas >= 30].index

# Filtrar el conjunto de datos para incluir solo estas empresas
empresas_interes = df[df['symbol'].isin(empresas_30_obs)]

empresas_interes.describe(include=object)

,date,symbol,reportedCurrency,fillingDate,acceptedDate,period,link,finalLink
count,44935,44935,44935,44935,44934,44935,37863,37859
unique,1134,821,15,3420,37708,5,37610,33983
top,2016-12-31 00:00:00,TILE,USD,2013-12-31 00:00:00,2013-12-31 00:00:00,Q1,https://www.sec.gov/Archives/edgar/data/165204...,https://www.sec.gov/Archives/edgar/data/165204...
freq,721,75,41498,188,188,11686,4,4


In [42]:
empresas_interes['date'] = pd.to_datetime(df['date'])

# Filtrar las fechas mínimas y máximas para cada empresa
min_fecha = empresas_interes.groupby('symbol')['date'].min()
max_fecha = empresas_interes.groupby('symbol')['date'].max()

# Filtrar las ventas iniciales y finales para cada empresa
inicio_ventas = empresas_interes.groupby('symbol').apply(lambda x: x[x['date'] == min_fecha[x.name]]['revenue'].iloc[0])
fin_ventas = empresas_interes.groupby('symbol').apply(lambda x: x[x['date'] == max_fecha[x.name]]['revenue'].iloc[0])

# Calcular el número de años entre las fechas mínimas y máximas
años = (max_fecha - min_fecha).dt.days / 365

# Calcular el CAGR de ventas para cada empresa
cagr_ventas = ((fin_ventas / inicio_ventas) ** (1 / años)) - 1

# Filtrar empresas con CAGR positivo
cagr_positivo = cagr_ventas[cagr_ventas > 0]

# Crear un DataFrame que contenga toda la información original de las empresas con CAGR positivo
cagr_positivo_df = empresas_interes[empresas_interes['symbol'].isin(cagr_positivo.index)]

cagr_positivo.head()

<ipython-input-42-e594754f633c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_interes['date'] = pd.to_datetime(df['date'])


symbol
AAL     0.035224
AAPL    0.209878
AAWW    0.075450
ABB     0.008507
ABCB    0.174854
dtype: float64

In [44]:
max_8_negativos = []

for symbol, group in empresas_interes.groupby('symbol'):
    try:
        conteo_negativos = (group['netIncome'] < 0).sum()
        if conteo_negativos <= 8:
            max_8_negativos.append([symbol, conteo_negativos])
    except:
        continue

mun_negativos = pd.DataFrame(max_8_negativos, columns=["symbol", "num_negativos_netIncome"])
print(mun_negativos)

# Crear un DataFrame que contenga toda la información original de las empresas
mun_negativos_df = empresas_interes[empresas_interes['symbol'].isin(mun_negativos.index)]

    symbol  num_negativos_netIncome
0     AAPL                        0
1      ABB                        1
2      ACN                        0
3     ADBE                        1
4      ADM                        0
..     ...                      ...
431     WU                        2
432    XOM                        4
433    XRX                        6
434   YNDX                        4
435     YY                        5

[436 rows x 2 columns]


In [45]:
# Filtrar las ventas iniciales y finales para cada empresa
inicio_costo = empresas_interes.groupby('symbol').apply(lambda x: x[x['date'] == min_fecha[x.name]]['costOfRevenue'].iloc[0])
fin_costo = empresas_interes.groupby('symbol').apply(lambda x: x[x['date'] == max_fecha[x.name]]['costOfRevenue'].iloc[0])

# Calcular el CAGR de ventas para cada empresa
cagr_costos = ((fin_costo / inicio_costo) ** (1 / años)) - 1

# Filter out companies where CAGR of revenue is greater than CAGR of expenses
diferencia = cagr_ventas - cagr_costos
mayor_cagr_ventas = diferencia[diferencia > 0]

mayor_cagr_ventas_df = empresas_interes[empresas_interes['symbol'].isin(mayor_cagr_ventas.index)]

mayor_cagr_ventas

symbol
AAPL    0.009591
AAWW    0.018916
ABB     0.001779
ABMD    0.009105
ACN     0.005891
          ...   
WTS     0.010711
WU      0.001352
X       0.018531
XENT    0.173497
ZEN     0.091048
Length: 345, dtype: float64

In [46]:
# Companies with positive CAGR
cagr_positivo_df = set(cagr_positivo_df.index)

# Companies with maximum 8 negative net income
mun_negativos_df = set(mun_negativos_df['symbol'])

# Companies with stable PE Ratio
mayor_cagr_ventas_df = set(mayor_cagr_ventas_df['symbol'])

# Intersect the companies that meet all three criteria
empresas_solidas = cagr_positivo_df & mun_negativos_df & mayor_cagr_ventas_df

# Create a DataFrame to store the results
empresas_a_invertir = pd.DataFrame(list(empresas_solidas), columns=['symbol'])


AttributeError: 'set' object has no attribute 'index'

In [39]:
empresas_solidas

set()